In [113]:
import geopandas as gpd
import geopy
from shapely.geometry import Point
from geopy.geocoders import Nominatim

In [114]:
import pandas as pd
import numpy as np
import mpu
import seaborn as sns

In [115]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 100)

In [116]:
# read excel file that has atms of bank misr with their [ID, Address, منطقة ,قسم / مركز ,محافظة, Lat, Long]
atm_df = pd.read_excel('Atms_bankmisr_AR.xlsx')
atm_df

,ID,Address,منطقة,قسم / مركز,محافظة,Lat,Long
0,1,Alfayoum Beghous Alhoriea St.,قسم رابع,قسم الفيوم,الفيوم,29.304412,30.853546
1,2,Fayoum University,قسم ثانى,قسم الفيوم,الفيوم,29.319696,30.835641
2,3,Fayoum AlHoriea St.,قسم اول,قسم الفيوم,الفيوم,29.308199,30.842229
3,4,Sanoras - Fayoum,مدينه سنورس,مركز سنورس,الفيوم,29.408575,30.865073
4,5,194 Abo Bakr AlSiddiq St.- Atsa City - Fayoum,مدينه اطسا,مركز أطسا,الفيوم,29.236966,30.790194
...,...,...,...,...,...,...,...
4258,4259,Suez Canal Authority Water Company,قسم ثالث,قسم الأربعين,السويس,29.967446,32.550833
4259,4260,K 36 Suez - Ain Sokhna Road,فنار وبير أبو الدرج,قسم عتاقة,السويس,29.728850,32.381612
4260,4261,Suez Sokhna Road Canary Village,فنار وبير أبو الدرج,قسم عتاقة,السويس,29.617352,32.317460
4261,4262,AlGeish St. Suez District Suez in front of Oma...,قسم ثالث,قسم الأربعين,السويس,29.967901,32.553257


In [117]:
# read all scrapped branches from bank misr website with their address, lat and long
branches_df = pd.read_excel('Branches_combinig/All_branches_uncleaned.xlsx')
branches_df.head()

,locations__card,locations__card__location,locations__card__phones,locations__card__phones href,locations__card__fax,locations__card__services,locations__card__direction-btn href,Unnamed: 7
0,Damanhour,Banque Misr Damanhour Branch - 1 Saad Zaghloul...,3318787- 045/3337107,tel:3318787- 045/3337107,045/3310345,8:30 AM - 3:00 PM,http://maps.google.com/maps?q=31.0424194780001...,NaN
1,Rashid,Albahr St. - Rasheed - Beside the school compl...,045/2921357,tel:045/2921357,045/2923452,8:30 AM - 3:00 PM,http://maps.google.com/maps?q=31.4063106180001...,NaN
2,Almahmoudia,Mahmoudia Canal St.,045/2500438,tel:045/2500438,045/2501375,8:30 AM - 3:00 PM,"http://maps.google.com/maps?q=31.182750578,30....",NaN
3,Abo Almatamir,Mohamed Metwally Alshaarawy St. - Abo Almatamir,045/2410222,tel:045/2410222,045/2400210,8:30 AM - 3:00 PM,http://maps.google.com/maps?q=30.9107890830001...,NaN
4,kafr Aldawr,Kafr Aldawar - Port Said St.,045/2212350,tel:045/2212350,045/2216835,8:30 AM - 3:00 PM,"http://maps.google.com/maps?q=31.133868218,30....",NaN


In [118]:
branches_df = branches_df[['locations__card__location','locations__card__direction-btn href']]
branches_df.head()

,locations__card__location,locations__card__direction-btn href
0,Banque Misr Damanhour Branch - 1 Saad Zaghloul...,http://maps.google.com/maps?q=31.0424194780001...
1,Albahr St. - Rasheed - Beside the school compl...,http://maps.google.com/maps?q=31.4063106180001...
2,Mahmoudia Canal St.,"http://maps.google.com/maps?q=31.182750578,30...."
3,Mohamed Metwally Alshaarawy St. - Abo Almatamir,http://maps.google.com/maps?q=30.9107890830001...
4,Kafr Aldawar - Port Said St.,"http://maps.google.com/maps?q=31.133868218,30...."


In [119]:
def get_lat_long(z):
    x = z[1]
    z['Lat'] = x[x.find('=')+1:x.find(',')]
    z['Long'] = x[x.find(',')+1:]
    return z
    

In [120]:
branches_df = branches_df.apply(get_lat_long,axis = 1)[['locations__card__location','Lat','Long']]
branches_df.rename(columns={'locations__card__location':'Branch_Address'},inplace=True)
branches_df

,Branch_Address,Lat,Long
0,Banque Misr Damanhour Branch - 1 Saad Zaghloul...,31.0424194780001,30.4689092350001
1,Albahr St. - Rasheed - Beside the school compl...,31.4063106180001,30.423861399
2,Mahmoudia Canal St.,31.182750578,30.5275566920001
3,Mohamed Metwally Alshaarawy St. - Abo Almatamir,30.9107890830001,30.1770190500001
4,Kafr Aldawar - Port Said St.,31.133868218,30.135129858
...,...,...,...
730,Arrival hall Suez Port,29.942901353,32.567301931
731,"Port Tawfiq in front of Gate No. (5), the cust...",29.942304,32.560966
732,Arrival visa office at the new airport adminis...,29.937393,32.557254
733,Arkan Mall - Sheikh Zayed Entrance 2 - in fron...,29.964206,32.54957


In [ ]:
# after cleaning the scrapped branches excel file
# branches_df.to_excel('Branches_Location.xlsx',index=0)

In [124]:
# using temp to avoid lossing runtime data by accident
temp = atm_df
temp.head()

,ID,Address,منطقة,قسم / مركز,محافظة,Lat,Long
0,1,Alfayoum Beghous Alhoriea St.,قسم رابع,قسم الفيوم,الفيوم,29.304412,30.853546
1,2,Fayoum University,قسم ثانى,قسم الفيوم,الفيوم,29.319696,30.835641
2,3,Fayoum AlHoriea St.,قسم اول,قسم الفيوم,الفيوم,29.308199,30.842229
3,4,Sanoras - Fayoum,مدينه سنورس,مركز سنورس,الفيوم,29.408575,30.865073
4,5,194 Abo Bakr AlSiddiq St.- Atsa City - Fayoum,مدينه اطسا,مركز أطسا,الفيوم,29.236966,30.790194


In [ ]:
def count_branches(x):
    lat1 = x[5]
    lon1 = x[6]
    distances = [] # hold the distance between single atm and all branches
    
    for i in range(len(branches_df)):
        lat2 = float(branches_df['Lat'].iloc[i])
        lon2 = float(branches_df['Long'].iloc[i])
        
        distances.append(mpu.haversine_distance((lat1, lon1), (lat2, lon2))*1000)
    
    min_distance = np.array(distances).min()
    index_min_distance = np.array(distances).argmin()
    
    x['Nearest_Branch_distance'] = min_distance
    x['Nearest_Branch_Address'] =  branches_df.iloc[index_min_distance]['Branch_Address']
    x['Lat_branch'] = branches_df.iloc[index_min_distance]['Lat']
    x['Long_branch'] = branches_df.iloc[index_min_distance]['Long']
    x['منطقة الفرع'] = branches_df.iloc[index_min_distance]['منطقة']
    x['قسم / مركز الفرع'] = branches_df.iloc[index_min_distance]['قسم / مركز']
    x['محافظة الفرع'] = branches_df.iloc[index_min_distance]['محافظة']
    

    return x
    

In [34]:
atm_df = atm_df[0:].apply(count_branches,axis = 1)

In [35]:
# this is all atms with their nearest branches
atm_df

,ID,Address,منطقة,قسم / مركز,محافظة,Lat,Long,Nearest_Branch_distance,Nearest_Branch_Address,Lat_branch,Long_branch,منطقة الفرع,قسم / مركز الفرع,محافظة الفرع
0,1,Alfayoum Beghous Alhoriea St.,قسم رابع,قسم الفيوم,الفيوم,29.304412,30.853546,0.000000,Alfayoum Beghous Alhoriea St.,29.304412,30.853546,قسم رابع,قسم الفيوم,الفيوم
1,2,Fayoum University,قسم ثانى,قسم الفيوم,الفيوم,29.319696,30.835641,1480.524070,Fayoum Security Directorate building,29.310533,30.846720,قسم ثانى,قسم الفيوم,الفيوم
2,3,Fayoum AlHoriea St.,قسم اول,قسم الفيوم,الفيوم,29.308199,30.842229,382.882978,AlFayoum 45 AlHorreya St. Fayoum,29.307723,30.846140,قسم اول,قسم الفيوم,الفيوم
3,4,Sanoras - Fayoum,مدينه سنورس,مركز سنورس,الفيوم,29.408575,30.865073,0.000000,Sanoras - Fayoum,29.408575,30.865073,مدينه سنورس,مركز سنورس,الفيوم
4,5,194 Abo Bakr AlSiddiq St.- Atsa City - Fayoum,مدينه اطسا,مركز أطسا,الفيوم,29.236966,30.790194,0.000000,194 Abo Bakr AlSiddiq St.- Atsa City - Fayoum,29.236966,30.790194,مدينه اطسا,مركز أطسا,الفيوم
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4258,4259,Suez Canal Authority Water Company,قسم ثالث,قسم الأربعين,السويس,29.967446,32.550833,338.241576,Corner of Talaat Harb St. and 23 July,29.970485,32.550995,قسم ثالث,قسم الأربعين,السويس
4259,4260,K 36 Suez - Ain Sokhna Road,فنار وبير أبو الدرج,قسم عتاقة,السويس,29.728850,32.381612,8968.540446,Egyptian-Chinese Joint Investment Corporation ...,29.670297,32.317752,فنار وبير أبو الدرج,قسم عتاقة,السويس
4260,4261,Suez Sokhna Road Canary Village,فنار وبير أبو الدرج,قسم عتاقة,السويس,29.617352,32.317460,5887.338247,Egyptian-Chinese Joint Investment Corporation ...,29.670297,32.317752,فنار وبير أبو الدرج,قسم عتاقة,السويس
4261,4262,AlGeish St. Suez District Suez in front of Oma...,قسم ثالث,قسم الأربعين,السويس,29.967901,32.553257,360.558005,Corner of Talaat Harb St. and 23 July,29.970485,32.550995,قسم ثالث,قسم الأربعين,السويس


# Results

In [73]:
atm_df[atm_df['Nearest_Branch_distance'] == 0].count()['ID']
# 452 atm in branches

452

In [74]:
atm_df[(atm_df['Nearest_Branch_distance'] > 0) & (atm_df['Nearest_Branch_distance'] <= 50)].count()['ID']
# 147 atm within the range of (0-50)m of their nearest branch

147

In [75]:
atm_df[(atm_df['Nearest_Branch_distance'] > 50) & (atm_df['Nearest_Branch_distance'] <= 100)].count()['ID']\
# 98 atm within the range of (50-100)m of their nearest branch

98

In [76]:
atm_df[(atm_df['Nearest_Branch_distance'] > 100) & (atm_df['Nearest_Branch_distance'] <= 150)].count()['ID']
# 108 atm within the range of (100-150)m of their nearest branch

108

In [77]:
atm_df[(atm_df['Nearest_Branch_distance'] > 150) & (atm_df['Nearest_Branch_distance'] <= 200)].count()['ID']
# 108 atm within the range of (150-200)m of their nearest branch

123

In [78]:
atm_df[(atm_df['Nearest_Branch_distance'] > 200) & (atm_df['Nearest_Branch_distance'] <= 250)].count()['ID']
# 108 atm within the range of (200-250)m of their nearest branch

150

In [79]:
# drop atms that are withn the range of (0-100)m of their nearest branch
# 100m wide is a personal choice as i think that these atms are there just to ease the crowd from the
# branch and doesn't have to be moved
modified_atm_df = atm_df.drop(atm_df[atm_df['Nearest_Branch_distance'] <=100 ].index)

In [86]:
# drop old id column
modified_atm_df.drop('ID',axis = 1, inplace = True)

In [87]:
# insert id column 
modified_atm_df.insert(0,'ID',range(1,len(temp_atm_df)+1))
modified_atm_df

,ID,Address,منطقة,قسم / مركز,محافظة,Lat,Long,Nearest_Branch_distance,Nearest_Branch_Address,Lat_branch,Long_branch,منطقة الفرع,قسم / مركز الفرع,محافظة الفرع
1,1,Fayoum University,قسم ثانى,قسم الفيوم,الفيوم,29.319696,30.835641,1480.524070,Fayoum Security Directorate building,29.310533,30.846720,قسم ثانى,قسم الفيوم,الفيوم
2,2,Fayoum AlHoriea St.,قسم اول,قسم الفيوم,الفيوم,29.308199,30.842229,382.882978,AlFayoum 45 AlHorreya St. Fayoum,29.307723,30.846140,قسم اول,قسم الفيوم,الفيوم
7,3,10th of Ramadan - Industrial Zone B1,قسم ثانى,قسم الفيوم,الفيوم,29.321487,30.837896,1488.462407,Fayoum Security Directorate building,29.310533,30.846720,قسم ثانى,قسم الفيوم,الفيوم
8,4,Faculty of Arts Fayoum University,قسم ثانى,قسم الفيوم,الفيوم,29.322145,30.837087,1593.570183,Fayoum Security Directorate building,29.310533,30.846720,قسم ثانى,قسم الفيوم,الفيوم
9,5,Fayoum train station,قسم اول,قسم الفيوم,الفيوم,29.308889,30.847499,184.899213,AlFayoum 45 AlHorreya St. Fayoum,29.307723,30.846140,قسم اول,قسم الفيوم,الفيوم
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4258,3562,Suez Canal Authority Water Company,قسم ثالث,قسم الأربعين,السويس,29.967446,32.550833,338.241576,Corner of Talaat Harb St. and 23 July,29.970485,32.550995,قسم ثالث,قسم الأربعين,السويس
4259,3563,K 36 Suez - Ain Sokhna Road,فنار وبير أبو الدرج,قسم عتاقة,السويس,29.728850,32.381612,8968.540446,Egyptian-Chinese Joint Investment Corporation ...,29.670297,32.317752,فنار وبير أبو الدرج,قسم عتاقة,السويس
4260,3564,Suez Sokhna Road Canary Village,فنار وبير أبو الدرج,قسم عتاقة,السويس,29.617352,32.317460,5887.338247,Egyptian-Chinese Joint Investment Corporation ...,29.670297,32.317752,فنار وبير أبو الدرج,قسم عتاقة,السويس
4261,3565,AlGeish St. Suez District Suez in front of Oma...,قسم ثالث,قسم الأربعين,السويس,29.967901,32.553257,360.558005,Corner of Talaat Harb St. and 23 July,29.970485,32.550995,قسم ثالث,قسم الأربعين,السويس


In [89]:
modified_atm_df.to_excel('ATM_Bank_Misr_except_in_branches.xlsx',index=0)

In [45]:
atm_df.to_excel('Is_near_branch.xlsx',index=0)